In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import os

import networkx as nx

import pickle
from PIL import Image, ImageDraw
import matplotlib.lines as lines
from mpl_toolkits import mplot3d
from matplotlib.lines import Line2D

from tqdm import tqdm
import glob

import statistics 

#For network adjacency checks
from skimage.transform import rotate
from sklearn.neighbors import KDTree
from scipy.spatial import distance

#For PCA
import pandas as pd
import seaborn as sns
import json
import re

import warnings

np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [5]:
%cd ..

/Users/trevorchan/research/morpho_scripts


In [263]:
with open('directed_graph/combineddf.data', 'rb') as pickle_file:
    combineddf = pickle.load(pickle_file)
with open('directed_graph/YMcombineddf.data', 'rb') as pickle_file:
    YMcombineddf = pickle.load(pickle_file)
with open('directed_graph/YMRcombineddf.data', 'rb') as pickle_file:
    YMRcombineddf = pickle.load(pickle_file)

corr = combineddf.corr()

links = corr.stack().reset_index()
links.columns = ['var1', 'var2', 'value']

# Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
links_filtered=links.loc[ (abs(links['value']) > 0.9) & (links['var1'] != links['var2']) ]

links_filtered = links_filtered.drop_duplicates(subset=['value'])


In [243]:
with open('directed_graph/cm_combined.data', 'rb') as pickle_file:
    cm = pickle.load(pickle_file)

corr = cm

links = corr.stack().reset_index()
links.columns = ['var1', 'var2', 'value']

# Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
links_filtered=links.loc[ (abs(links['value']) > 0.3) & (links['var1'] != links['var2']) ]

links_filtered = links_filtered.drop_duplicates(subset=['value'])


In [244]:
# #Use for showing only 3D to 2D correlations

# twod = ['ncompk', 'mdeg_l', 'dmdeg_l', 'mdeg_h', 'dmdeg_h', 
#                 'fdim0',  'cnumh', 'cnumk','circum_h','area_l', 'circum_l', 'areavar','circumvar','asravar', 'circ1', 'asra1', 
#                 'area_h', 'circ0', 'asra0', 'circvar',]
# threed = ['protrusion_mean_len', 'protrusion_max_len','protrusion_num','discr_mean_dist', 'discr_max_dist', 'discr_mean_area', 
#       'discr_num','area', 'perimeter', 'aspect_ratio', 'roundness', 'circularity',]

# pop_index = []
# for index, row in links_filtered.iterrows():
    
#     if row['var1'] in twod:
#         if row['var2'] in threed:
#             continue
#     if row['var1'] in threed:
#         if row['var2'] in twod:
#             continue
#     pop_index.append(index)
# links_filtered = links_filtered.drop(pop_index)

In [245]:
unique_vars = pd.DataFrame(np.unique(corr.columns))

In [246]:
cm.columns = [ 'ncomph', 	'ncompk', 	'mdeg_l', 	'dmdeg_l', 	'mdeg_h', 	'dmdeg_h', 	'mcompmass_h', 	     
        'mcompmass_d', 	'fdim0', 'circum_l', 	'circum_h', 	'circ0', 	'circ1', 	'asra0', 	'asra1', 	
        'areavar', 	'circumvar', 	 'circvar', 	'asravar', 'cnumh',   'cnumk',   'area_l',   'area_h' ,  
        '3D invasiveness']

cm.index = [ 'ncomph', 	'ncompk', 	'mdeg_l', 	'dmdeg_l', 	'mdeg_h', 	'dmdeg_h', 	'mcompmass_h', 	     
        'mcompmass_d', 	'fdim0', 'circum_l', 	'circum_h', 	'circ0', 	'circ1', 	'asra0', 	'asra1', 	
        'areavar', 	'circumvar', 	 'circvar', 	'asravar', 'cnumh',   'cnumk',   'area_l',   'area_h' ,  
        '3D invasiveness']
# cm = cm.rename(index={0: 'ncomph', 	1:'ncompk', 	2:'mdeg_l', 	3:'dmdeg_l', 	4:'mdeg_h', 	5:'dmdeg_h', 	6:'mcompmass_h', 	     
#         7:'mcompmass_d', 	8:'fdim0',9: 'circum_l', 	10:'circum_h', 11:	'circ0', 	12:'circ1', 	13:'asra0', 	14:'asra1', 	
#         15:'areavar', 	16:'circumvar', 	 17:'circvar', 	18:'asravar', 19:'cnumh',   20:'cnumk',   21:'area_l',   22:'area_h' ,  
#         23:'3D invasiveness'})

cm

,ncomph,ncompk,mdeg_l,dmdeg_l,mdeg_h,dmdeg_h,mcompmass_h,mcompmass_d,fdim0,circum_l,...,asra1,areavar,circumvar,circvar,asravar,cnumh,cnumk,area_l,area_h,3D invasiveness
ncomph,1.000000,0.359401,-0.172174,-0.607720,-0.536225,-0.142936,-0.140597,-0.103748,0.501656,-0.069519,...,-0.480816,-0.393421,0.539896,0.389101,-0.556659,-0.541822,-0.510758,-0.253073,-0.457373,0.259424
ncompk,0.359401,1.000000,-0.943529,-0.887062,-0.824461,0.615167,-0.689334,-0.073877,-0.111068,0.173316,...,0.377310,0.323321,-0.136438,-0.358992,0.088869,0.118939,0.172178,0.306411,0.103974,-0.388202
mdeg_l,-0.172174,-0.943529,1.000000,0.716890,0.728080,-0.700596,0.629688,-0.057424,0.226606,-0.310829,...,-0.505312,-0.547392,0.411001,0.558370,-0.345236,-0.329443,-0.419329,-0.551625,-0.343868,0.512567
dmdeg_l,-0.607720,-0.887062,0.716890,1.000000,0.923656,-0.255532,0.537082,0.086388,0.073767,0.000234,...,-0.082651,-0.032491,-0.167563,0.077000,0.206167,0.071835,0.083854,0.033064,0.159301,0.196443
mdeg_h,-0.536225,-0.824461,0.728080,0.923656,1.000000,-0.076080,0.554705,0.081923,0.234031,-0.022420,...,0.007663,-0.129268,0.015054,0.118886,0.096093,-0.131197,-0.124175,-0.116468,0.032650,0.346109
dmdeg_h,-0.142936,0.615167,-0.700596,-0.255532,-0.076080,1.000000,-0.355220,0.135536,0.033707,0.363591,...,0.737029,0.519176,-0.359386,-0.589449,0.426310,0.143741,0.261007,0.488879,0.359525,-0.255284
mcompmass_h,-0.140597,-0.689334,0.629688,0.537082,0.554705,-0.355220,1.000000,0.689163,-0.172775,0.237204,...,-0.054011,0.053169,-0.108839,-0.098440,0.233965,0.115580,0.067571,-0.166317,0.221511,0.327171
mcompmass_d,-0.103748,-0.073877,-0.057424,0.086388,0.081923,0.135536,0.689163,1.000000,-0.522654,0.515323,...,0.273099,0.444356,-0.438640,-0.511264,0.510003,0.387673,0.413266,0.214248,0.507195,0.155558
fdim0,0.501656,-0.111068,0.226606,0.073767,0.234031,0.033707,-0.172775,-0.522654,1.000000,-0.195925,...,-0.437044,-0.589787,0.638202,0.581219,-0.562777,-0.791771,-0.726407,-0.287181,-0.521911,0.409445
circum_l,-0.069519,0.173316,-0.310829,0.000234,-0.022420,0.363591,0.237204,0.515323,-0.195925,1.000000,...,0.277905,0.477119,-0.534508,-0.613560,0.691751,0.549823,0.590899,0.610853,0.837017,-0.123602


In [264]:
# templinks = [{'source':link[0],'target':link[1],'value':link[2]} for link in links_filtered.to_numpy()]

# links_list = []
# for link in templinks:
# #      unique_vars.index[unique_vars[0]=='solidity'].to_list()[0]
#     record = {"value":link['value'], "source": unique_vars.index[unique_vars[0]==link['source']].to_list()[0],
#              "target": unique_vars.index[unique_vars[0]==link['target']].to_list()[0]}
#     links_list.append(record)

IndexError: list index out of range

In [248]:
translate = {0: 'ncomph', 	1:'ncompk', 	2:'mdeg_l', 	3:'dmdeg_l', 	4:'mdeg_h', 	5:'dmdeg_h', 	6:'mcompmass_h', 	     
         7:'mcompmass_d', 	8:'fdim0',9: 'circum_l', 	10:'circum_h', 11:	'circ0', 	12:'circ1', 	13:'asra0', 	14:'asra1', 	
         15:'areavar', 	16:'circumvar', 	 17:'circvar', 	18:'asravar', 19:'cnumh',   20:'cnumk',   21:'area_l',   22:'area_h' ,  
         23:'3D invasiveness'}

In [265]:
templinks = [{'source':link[0],'target':link[1],'value':link[2]} for link in links_filtered.to_numpy()]

for i in range(len(templinks)):
    templinks[i]['source'] = translate[templinks[i]['source']]

links_list = []
for link in templinks:
#      unique_vars.index[unique_vars[0]=='solidity'].to_list()[0]
    record = {"value":link['value'], "source": unique_vars.index[unique_vars[0]==link['source']].to_list()[0],
             "target": unique_vars.index[unique_vars[0]==link['target']].to_list()[0]}
    links_list.append(record)

KeyError: 'ncompk'

In [256]:
links_list = [link for link in links_list if link['source']==0 or link['target']==0]
links_list

[{'value': -0.38820223321239283, 'source': 23, 'target': 0},
 {'value': 0.512566746030917, 'source': 21, 'target': 0},
 {'value': 0.3461092317542502, 'source': 20, 'target': 0},
 {'value': 0.3271708116203842, 'source': 19, 'target': 0},
 {'value': 0.4094454760617532, 'source': 17, 'target': 0},
 {'value': -0.5291401486884744, 'source': 7, 'target': 0},
 {'value': -0.6562014076675202, 'source': 8, 'target': 0},
 {'value': -0.5542238383422303, 'source': 4, 'target': 0},
 {'value': -0.6366872669047047, 'source': 5, 'target': 0},
 {'value': -0.7678844124891802, 'source': 3, 'target': 0},
 {'value': 0.7115150351924132, 'source': 11, 'target': 0},
 {'value': 0.6812270866249374, 'source': 12, 'target': 0},
 {'value': -0.6035556186747814, 'source': 6, 'target': 0},
 {'value': -0.704358439076123, 'source': 13, 'target': 0},
 {'value': -0.7068107604411011, 'source': 14, 'target': 0},
 {'value': -0.5780936393762367, 'source': 2, 'target': 0},
 {'value': -0.5077689245993596, 'source': 1, 'target':

In [257]:
#label colors and edges
def grouper(var):
    if var in ['ncompk', 'mdeg_l', 'dmdeg_l', 'mdeg_h', 'dmdeg_h', 
                'fdim0',  'cnumh', 'cnumk', ]:#Network Variables
        return '0'
    elif var in ['circum_h','area_l', 'circum_l', 'areavar','circumvar','asravar', 'circ1', 'asra1', 
                'area_h', 'circ0', 'asra0', 'circvar',]:#Single Cell Morph Vars
        return '1'
    elif var in [ ]:#Single Cell Morph Vars
        return '2'
    elif var in ['protrusion_mean_len', 'protrusion_max_len','protrusion_num',]:
        return '3'#Spheroid Protrusion Vars
    elif var in ['discr_mean_dist', 'discr_max_dist', 'discr_mean_area', 'discr_num']:
        return '4'#Spheroid Discrete Region Vars
    elif var in ['area', 'perimeter', 'aspect_ratio', 'roundness', 'circularity',]:
        return '5'#Random Spheroid Vars
    elif var in ['3D invasiveness']:
        return '5'
        

nodes_list = []
for var in unique_vars.to_numpy():
    nodes_list.append({'name':var[0],'group':grouper(var)})

In [258]:
json_prep = {"links":links_list, "nodes":nodes_list}

json_prep.keys()

dict_keys(['links', 'nodes'])

In [259]:
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [260]:
filename_out = 'directed_graph/pcap_export1.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()

In [261]:
len(links_filtered)

185

In [262]:
links_list

[{'value': -0.38820223321239283, 'source': 23, 'target': 0},
 {'value': 0.512566746030917, 'source': 21, 'target': 0},
 {'value': 0.3461092317542502, 'source': 20, 'target': 0},
 {'value': 0.3271708116203842, 'source': 19, 'target': 0},
 {'value': 0.4094454760617532, 'source': 17, 'target': 0},
 {'value': -0.5291401486884744, 'source': 7, 'target': 0},
 {'value': -0.6562014076675202, 'source': 8, 'target': 0},
 {'value': -0.5542238383422303, 'source': 4, 'target': 0},
 {'value': -0.6366872669047047, 'source': 5, 'target': 0},
 {'value': -0.7678844124891802, 'source': 3, 'target': 0},
 {'value': 0.7115150351924132, 'source': 11, 'target': 0},
 {'value': 0.6812270866249374, 'source': 12, 'target': 0},
 {'value': -0.6035556186747814, 'source': 6, 'target': 0},
 {'value': -0.704358439076123, 'source': 13, 'target': 0},
 {'value': -0.7068107604411011, 'source': 14, 'target': 0},
 {'value': -0.5780936393762367, 'source': 2, 'target': 0},
 {'value': -0.5077689245993596, 'source': 1, 'target':